In [30]:
import tweepy
import pandas as pd
import numpy as np
import pyodbc
import secret

In [31]:
#accessing the Twitter API
client = tweepy.Client(bearer_token = secret.BEARER)

In [32]:
#specifying query for searching Twitter
query = 'Wordle 6 -is:retweet'

#searching tweets per the query
tweets = client.search_recent_tweets(query=query,max_results= 25, tweet_fields=['author_id', 'created_at','geo'])

In [33]:
data = pd.DataFrame(data = [tweet.id for tweet in tweets.data], columns =['ID'])
data['Author'] = np.array([tweet.author_id for tweet in tweets.data])

data.head(10)

,ID,Author
0,1521296523673636865,909748323208257539
1,1521296514898874372,291483945
2,1521296512009052167,1651566632
3,1521296509291188226,161161255
4,1521296509257789440,2162991570
5,1521296491373109249,2424246505
6,1521296491104591872,238472629
7,1521296484456620036,338945841
8,1521296482091286528,174525578
9,1521296467469606913,1149219321182244865


In [34]:
score_arr = []
day_arr = []
isHard_arr = []

for tweet in tweets.data:
    score = tweet.text.find("/")-1

    if tweet.text[score].isnumeric():
        score_arr.append(tweet.text[score])
    else:
        score_arr.append(0)

    temp_day = tweet.text.partition("Wordle ")
    day = temp_day[2][0:4]

    if(day[0:3].isnumeric() and day[3] == " "):
        day_arr.append(day[0:3])
    else:
        day_arr.append('NaN')

    if(tweet.text[score+3] == '*'):
        isHard_arr.append(1)
    else:
        isHard_arr.append(0)

In [35]:
data['WordleDay'] = day_arr
data['Score'] = score_arr
data['HardMode'] = isHard_arr

In [36]:
clean_data = data[data['WordleDay'].str.contains("NaN") == False]

clean_data.head(10)

,ID,Author,WordleDay,Score,HardMode
0,1521296523673636865,909748323208257539,318,4,0
1,1521296514898874372,291483945,317,2,0
3,1521296509291188226,161161255,317,5,0
4,1521296509257789440,2162991570,318,4,0
5,1521296491373109249,2424246505,318,5,0
6,1521296491104591872,238472629,317,5,0
7,1521296484456620036,338945841,318,3,0
8,1521296482091286528,174525578,318,5,0
9,1521296467469606913,1149219321182244865,318,3,0
10,1521296446422765568,121637897,317,3,0


In [37]:
#uploading dataframe to the databse
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=AUSTIN-PC\SQLEXPRESS;'
                      'Database=WordleAnalysis;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

for index, row in clean_data.iterrows():
    cursor.execute("INSERT INTO TwitterWordle (ID,Author,WordleDay,Score,HardMode) values(?,?,?,?,?)", row.ID, row.Author, row.WordleDay, row.Score, row.HardMode)
conn.commit()
cursor.close()